# Load data from energinet.dk
Contributors: Karen Olsen<br>
Purpose: Load power generation and consumption in xml format, combine and save as dataframe.


In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET 
import dateutil.parser
import datetime as dt
import windpype as wp

windpype submodule "power" imported
windpype submodule "aux" imported
windpype submodule "capacity" imported
windpype module import complete


## Load power production data, 1 hour resolution

In [2]:
file_path = "data/Energinet/1hour/"
power_ob1 = wp.PowerData() # for Electricity Balance Data
df_cols1 = ['HourDK', 'PriceArea', 'OffshoreWindPower','OnshoreWindPower','SolarPower']
col_indices1 = [1,2,6,7,10]
raw_data_names1 = ["Electricity_Balance_Data_2011-2018_DK1.xml","Electricity_Balance_Data_2011-2018_DK2.xml"]
power_ob1.AddData(file_path=file_path,raw_data_names=raw_data_names1,df_cols=df_cols1,col_indices=col_indices1,\
                  time='HourDK')
power_ob1.FillNansNULLS(col_name='OffshoreWindPower')
power_ob1.FillNansNULLS(col_name='OnshoreWindPower')
power_ob1.FillNansNULLS(col_name='SolarPower')

Loading xml file at data/Energinet/1hour/Electricity_Balance_Data_2011-2018_DK1.xml
Number of datapoints: 67919
could not make column PriceArea into float, type: <class 'str'>
made column OffshoreWindPower into float
made column OnshoreWindPower into float
could not make column SolarPower into float, type: <class 'str'>
Loading xml file at data/Energinet/1hour/Electricity_Balance_Data_2011-2018_DK2.xml
Number of datapoints: 67919
could not make column PriceArea into float, type: <class 'str'>
made column OffshoreWindPower into float
made column OnshoreWindPower into float
could not make column SolarPower into float, type: <class 'str'>
Using method: append
Length of dataframe 1: 67919
Length of dataframe 2: 67919
Combined data over common time period from 2011-01-01 00:00:00 to 2018-09-30 23:00:00
135838 datapoints
Replacing NULL in 0 places with 0 in OffshoreWindPower
Replacing nans in 0 places with 0 in OffshoreWindPower


/home/karen/Projects/EPIMES/python/windpype_dev/windpype/power.py:2606: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  N_places = len(array[array == 'NULL'])
/home/karen/Projects/EPIMES/python/windpype_dev/windpype/power.py:2612: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  array[array == 'NULL'] = 0


Replacing NULL in 0 places with 0 in OnshoreWindPower
Replacing nans in 0 places with 0 in OnshoreWindPower
Replacing NULL in 51936 places with 0 in SolarPower
Replacing nans in 0 places with 0 in SolarPower


In [3]:
power_ob1.SplitData('PriceArea','DK1','DK2')
array = power_ob1.data_df['OnshoreWindPower'].values + power_ob1.data_df['OffshoreWindPower'].values
power_ob1.AddColumnToData('TotalWindPower',array)
array = power_ob1.data_df['OnshoreWindPower_DK1'].values + power_ob1.data_df['OffshoreWindPower_DK1'].values
power_ob1.AddColumnToData('TotalWindPower_DK1',array)
array = power_ob1.data_df['OnshoreWindPower_DK2'].values + power_ob1.data_df['OffshoreWindPower_DK2'].values
power_ob1.AddColumnToData('TotalWindPower_DK2',array)
array = power_ob1.data_df['OnshoreWindPower_DK1'].values + power_ob1.data_df['OnshoreWindPower_DK2'].values
power_ob1.AddColumnToData('TotalOnshoreWindPower',array)
array = power_ob1.data_df['OffshoreWindPower_DK1'].values + power_ob1.data_df['OffshoreWindPower_DK2'].values
power_ob1.AddColumnToData('TotalOffshoreWindPower',array)
# Renewable generation
array = power_ob1.data_df['SolarPower'].values + power_ob1.data_df['TotalWindPower'].values
power_ob1.AddColumnToData('TotalRenPower',array)
array = power_ob1.data_df['SolarPower_DK1'].values + power_ob1.data_df['TotalWindPower_DK1'].values
power_ob1.AddColumnToData('TotalRenPower_DK1',array)
array = power_ob1.data_df['SolarPower_DK2'].values + power_ob1.data_df['TotalWindPower_DK2'].values
power_ob1.AddColumnToData('TotalRenPower_DK2',array)

Added two new columns for OffshoreWindPower, with DK1 and DK2 partitions
Added two new columns for OnshoreWindPower, with DK1 and DK2 partitions
Added two new columns for SolarPower, with DK1 and DK2 partitions
New dataframe has 67919 rows


In [4]:
power_ob1.RemoveCrazyValues(col_name='TotalWindPower',method='above',value=10000)
power_ob1.RemoveDuplicateTimes(col_name='TotalWindPower')
power_ob1.FillNansNULLS(col_name='TotalWindPower')
power_ob1.FillNansNULLS(col_name='SolarPower')
power_ob1.FillInTimeSteps()
wp.aux.SaveData(power_ob1.data_df,file_path='data/',file_name='DK1_DK2_power_1hour')
power_ob1.info()

Replacing 0 values in TotalWindPower


/home/karen/Projects/EPIMES/python/windpype_dev/windpype/power.py:2573: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data_df[key][list(indices_to_replace)] = (data_df[key][list(indices_to_replace-1)].values + data_df[key][list(indices_to_replace+1)].values)/2.


Found duplicates at:
2011-10-30 02:00:00
2012-10-28 02:00:00
2013-10-27 02:00:00
2014-10-26 02:00:00
2015-10-25 02:00:00
2016-10-30 02:00:00
2017-10-29 02:00:00
Replacing NULL in 0 places with 0 in TotalWindPower
Replacing nans in 0 places with 0 in TotalWindPower


/home/karen/Projects/EPIMES/python/windpype_dev/windpype/power.py:2606: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  N_places = len(array[array == 'NULL'])
/home/karen/Projects/EPIMES/python/windpype_dev/windpype/power.py:2612: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  array[array == 'NULL'] = 0


Replacing NULL in 0 places with 0 in SolarPower
Replacing nans in 0 places with 0 in SolarPower


/home/karen/Projects/EPIMES/python/windpype_dev/windpype/power.py:2548: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data_df_temp['datetime'][i] = new_datetime


Adding 18 new rows of data
Using method: merge
Length of dataframe 1: 67912
Length of dataframe 2: 18
Combined data over common time period from 2011-01-01 00:00:00 to 2018-09-30 23:00:00
67920 datapoints

--------
Data object contains:
67920 data points
from 2011-01-01 00:00:00 to 2018-09-30 23:00:00
Minimum time step: 3600.0 sec
Maximum time step: 3600.0 sec
Most common time step: 3600.0 sec
--------


(Timestamp('2011-01-01 00:00:00'), Timestamp('2018-09-30 23:00:00'))

## Add consumption data, 1 hour resolution

In [5]:
file_path = "../../data/Energinet/1hour/"
power_ob2 = wp.PowerData() # for Electricity Balance Data
df_cols1 = ['Minutes5DK', 'PriceArea', 'GrossCon']
col_indices1 = [1,2,3]
raw_data_names1 = ["Electricity_Balance_Data_2013-2018.xml"]
power_ob2.AddData(file_path=file_path,raw_data_names=raw_data_names1,df_cols=df_cols1,col_indices=col_indices1,time='Minutes5DK')
power_ob2.FillNansNULLS(col_name='GrossCon')
power_ob2.SplitData('PriceArea','DK1','DK2')
power_ob2.info()

Loading xml file at ../../data/Energinet/1hour/Electricity_Balance_Data_2013-2018.xml
Number of datapoints: 87650
could not make column PriceArea into float, type: <class 'str'>
could not make column GrossCon into float, type: <class 'str'>
Replacing NULL in 2 places with 0 in GrossCon
Replacing nans in 0 places with 0 in GrossCon
Added two new columns for GrossCon, with DK1 and DK2 partitions
New dataframe has 43825 rows

--------
Data object contains:
43825 data points
from 2013-09-01 00:00:00 to 2018-09-01 00:00:00
Minimum time step: 0.0 sec
Maximum time step: 7200.0 sec
Most common time step: 3600.0 sec
--------


(Timestamp('2013-09-01 00:00:00'), Timestamp('2018-09-01 00:00:00'))

Combine and save data

In [6]:
power_ob = wp.CombPowerData(ob_1=power_ob1,ob_2=power_ob2,method='merge')

Using method: merge
Length of dataframe 1: 67920
Length of dataframe 2: 43825
Combined data over common time period from 2011-01-01 00:00:00 to 2018-09-30 23:00:00
67925 datapoints


In [10]:
# Residual load
array = power_ob.data_df['GrossCon'].values - power_ob.data_df['TotalRenPower'].values
power_ob.AddColumnToData('TotalResLoad',array)
array = power_ob.data_df['GrossCon_DK1'].values - power_ob.data_df['TotalRenPower_DK1'].values
power_ob.AddColumnToData('TotalResLoad_DK1',array)
array = power_ob.data_df['GrossCon_DK2'].values - power_ob.data_df['TotalRenPower_DK2'].values
power_ob.AddColumnToData('TotalResLoad_DK2',array)
power_ob.RemoveDuplicateTimes(col_name='datetime')
power_ob.info()

Column TotalResLoad already exists in dataframe
Column TotalResLoad_DK1 already exists in dataframe
Column TotalResLoad_DK2 already exists in dataframe
Found duplicates at:
2013-10-27 02:00:00
2014-10-26 02:00:00
2015-10-25 02:00:00
2016-10-30 02:00:00
2017-10-29 02:00:00

--------
Data object contains:
67920 data points
from 2011-01-01 00:00:00 to 2018-09-30 23:00:00
Minimum time step: 3600.0 sec
Maximum time step: 3600.0 sec
Most common time step: 3600.0 sec
--------


(Timestamp('2011-01-01 00:00:00'), Timestamp('2018-09-30 23:00:00'))

In [11]:
wp.aux.SaveData(power_ob.data_df,file_path='data/',file_name='DK1_DK2_power_cons_1hour')
power_ob.data_df.head()

,datetime,OffshoreWindPower_DK1,OffshoreWindPower_DK2,OffshoreWindPower,OnshoreWindPower_DK1,OnshoreWindPower_DK2,OnshoreWindPower,SolarPower_DK1,SolarPower_DK2,SolarPower,...,TotalRenPower_DK1,TotalRenPower_DK2,GrossCon,GrossCon_DK1,GrossCon_DK2,delta_time,time_steps,TotalResLoad,TotalResLoad_DK1,TotalResLoad_DK2
0,2011-01-01 00:00:00,618.093464,204.306536,822.4,1926.581694,636.818306,2563.4,0.0,0.0,0.0,...,2544.675157,841.124843,-1.0,-1.0,-1.0,0.0,3600.0,-3386.8,-2545.675157,-842.124843
1,2011-01-01 01:00:00,621.428703,208.871297,830.3,1883.371297,633.028703,2516.4,0.0,0.0,0.0,...,2504.800000,841.900000,-1.0,-1.0,-1.0,3600.0,3600.0,-3347.7,-2505.800000,-842.900000
2,2011-01-01 02:00:00,615.024796,214.975204,830.0,1760.675204,615.424796,2376.1,0.0,0.0,0.0,...,2375.700000,830.400000,-1.0,-1.0,-1.0,7200.0,3600.0,-3207.1,-2376.700000,-831.400000
3,2011-01-01 03:00:00,613.623262,215.176738,828.8,1744.176738,611.623262,2355.8,0.0,0.0,0.0,...,2357.800000,826.800000,-1.0,-1.0,-1.0,10800.0,3600.0,-3185.6,-2358.800000,-827.800000
4,2011-01-01 04:00:00,609.550003,211.949997,821.5,1749.849997,608.450003,2358.3,0.0,0.0,0.0,...,2359.400000,820.400000,-1.0,-1.0,-1.0,14400.0,3600.0,-3180.8,-2360.400000,-821.400000
